In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_ollama import ChatOllama

memory = SqliteSaver.from_conn_string(":memory:")

In [2]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
import os
API_KEY = "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"
os.environ["TAVILY_API_KEY"] = API_KEY

tavily_search_tool = TavilySearchResults(max_results=2)


In [21]:
tavily_search_tool.invoke("What is the weather in SF?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741438285, 'localtime': '2025-03-08 04:51'}, 'current': {'last_updated_epoch': 1741437900, 'last_updated': '2025-03-08 04:45', 'temp_c': 6.1, 'temp_f': 43.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 167, 'wind_dir': 'SSE', 'pressure_mb': 1024.0, 'pressure_in': 30.25, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 0, 'feelslike_c': 5.9, 'feelslike_f': 42.7, 'windchill_c': 6.5, 'windchill_f': 43.7, 'heatindex_c': 7.8, 'heatindex_f': 46.0, 'dewpoint_c': 7.3, 'dewpoint_f': 45.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 3.8, 'gust_kph': 6.1}}"},
 {'url': 'https://www.weather25.com/north-america/usa/c

In [5]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [31]:
from langgraph.checkpoint.memory import MemorySaver

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(model="llama3.1:8b")

memorySaver = MemorySaver()
abot = Agent(model, [tavily_search_tool], system=prompt, checkpointer=memorySaver)

In [32]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='a627207b-7f69-4d43-b85a-d822a6163cef'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T12:53:38.29077Z', 'done': True, 'done_reason': 'stop', 'total_duration': 593227791, 'load_duration': 27397250, 'prompt_eval_count': 259, 'prompt_eval_duration': 156000000, 'eval_count': 23, 'eval_duration': 408000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-45c7a1d6-e053-455e-a89b-0a42c1f6e21a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'd8867c91-ece8-4abc-b7b1-b10b8bdb3cd9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T12:53:38.29077Z

In [44]:
print(thread)
dir(abot.graph.get_state(thread))

{'configurable': {'thread_id': '1'}}


['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__reduce__',
 '__reduce_ex__',
 '__replace__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'config',
 'count',
 'created_at',
 'index',
 'metadata',
 'next',
 'parent_config',
 'tasks',
 'values']

In [49]:
abot.graph.get_state(thread)[:]

({'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='a627207b-7f69-4d43-b85a-d822a6163cef'),
   AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T12:53:38.29077Z', 'done': True, 'done_reason': 'stop', 'total_duration': 593227791, 'load_duration': 27397250, 'prompt_eval_count': 259, 'prompt_eval_duration': 156000000, 'eval_count': 23, 'eval_duration': 408000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-45c7a1d6-e053-455e-a89b-0a42c1f6e21a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'd8867c91-ece8-4abc-b7b1-b10b8bdb3cd9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282})]},
 ('action',),
 {'configurable': {'thread_id': '1',
   'checkpoint_ns': '',
   'checkpoint_id': '1effc1c5-ab5b-6c06-8001-e38b61dded51

In [58]:
abot.graph.get_state(thread).next

('action',)

In [59]:
# pass None to the stream to continue from the interrupted state
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'd8867c91-ece8-4abc-b7b1-b10b8bdb3cd9', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1741439550, \'localtime\': \'2025-03-08 05:12\'}, \'current\': {\'last_updated_epoch\': 1741438800, \'last_updated\': \'2025-03-08 05:00\', \'temp_c\': 6.7, \'temp_f\': 44.1, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 3.1, \'wind_kph\': 5.0, \'wind_degree\': 140, \'wind_dir\': \'SE\', \'pressure_mb\': 1024.0, \'pressure_in\': 30.25, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 85, \'cloud\': 25,

In [64]:
abot.graph.get_state(thread).values["messages"]

[HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='a627207b-7f69-4d43-b85a-d822a6163cef'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T12:53:38.29077Z', 'done': True, 'done_reason': 'stop', 'total_duration': 593227791, 'load_duration': 27397250, 'prompt_eval_count': 259, 'prompt_eval_duration': 156000000, 'eval_count': 23, 'eval_duration': 408000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-45c7a1d6-e053-455e-a89b-0a42c1f6e21a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'd8867c91-ece8-4abc-b7b1-b10b8bdb3cd9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282}),
 ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\

In [65]:
abot.graph.get_state(thread).next

()

In [66]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='e79a25d3-134b-47bb-a4d1-faa3137ac687'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:16:01.803955Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4046593500, 'load_duration': 31417833, 'prompt_eval_count': 259, 'prompt_eval_duration': 3601000000, 'eval_count': 23, 'eval_duration': 410000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a76b6f8d-3600-4e0e-8117-19417ebcce08-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather'}, 'id': 'f2abc782-0fd4-4bb6-8801-5b188177065c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:16:01.80395

Run until interrupt, then modify the state

In [67]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}

for event in abot.graph.stream({"messages": messages}, thread): 
    for v in event.values():
        print(v)


{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='36430960-e756-4687-912a-f53875802049'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2193879000, 'load_duration': 73929791, 'prompt_eval_count': 259, 'prompt_eval_duration': 1675000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-361026ce-1af4-45a7-8c5b-f8fa77fd9ef8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281})]}
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'don

In [69]:
curr_values = abot.graph.get_state(thread)

In [72]:
curr_values.values["messages"][:]


[HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='36430960-e756-4687-912a-f53875802049'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2193879000, 'load_duration': 73929791, 'prompt_eval_count': 259, 'prompt_eval_duration': 1675000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-361026ce-1af4-45a7-8c5b-f8fa77fd9ef8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281})]

In [73]:
curr_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LA weather'},
  'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00',
  'type': 'tool_call'}]

In [78]:
_id = curr_values.values['messages'][-1].tool_calls[0]['id']
print(_id)
# modify the last message
curr_values.values["messages"][-1].tool_calls = [
    {
        'name': 'tavily_search_results_json',
        'id': _id,
        'args': {
            'query': 'weather in Delhi, India'
        }
    }
]



c65bf393-ff29-4e2a-b27f-64e545737e00


In [81]:
abot.graph.get_state(thread).values["messages"][:]


[HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='36430960-e756-4687-912a-f53875802049'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2193879000, 'load_duration': 73929791, 'prompt_eval_count': 259, 'prompt_eval_duration': 1675000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-361026ce-1af4-45a7-8c5b-f8fa77fd9ef8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LA weather'}, 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281})]

In [83]:
abot.graph.update_state(thread, curr_values.values)

abot.graph.get_state(thread).values["messages"][:]

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='36430960-e756-4687-912a-f53875802049'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2193879000, 'load_duration': 73929791, 'prompt_eval_count': 259, 'prompt_eval_duration': 1675000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-361026ce-1af4-45a7-8c5b-f8fa77fd9ef8-0', tool_calls=[{'name': 'tavily_search_results_json', 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'args': {'query': 'weather in Delhi, India'}}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281})]}


[HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='36430960-e756-4687-912a-f53875802049'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-08T13:17:25.513351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2193879000, 'load_duration': 73929791, 'prompt_eval_count': 259, 'prompt_eval_duration': 1675000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-361026ce-1af4-45a7-8c5b-f8fa77fd9ef8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Delhi, India'}, 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 22, 'total_tokens': 281})]

In [84]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Delhi, India'}, 'id': 'c65bf393-ff29-4e2a-b27f-64e545737e00', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Delhi\', \'region\': \'Delhi\', \'country\': \'India\', \'lat\': 28.6667, \'lon\': 77.2167, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoch\': 1741440774, \'localtime\': \'2025-03-08 19:02\'}, \'current\': {\'last_updated_epoch\': 1741440600, \'last_updated\': \'2025-03-08 19:00\', \'temp_c\': 26.1, \'temp_f\': 79.0, \'is_day\': 0, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/143.png\', \'code\': 1030}, \'wind_mph\': 5.1, \'wind_kph\': 8.3, \'wind_degree\': 288, \'wind_dir\': \'WNW\', \'pressure_mb\': 1009.0, \'pressure_in\': 29.8, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 32, \'cloud\': 0, \'feelslike_c\': 24.5, \'feelslike_f\': 76.2, 